In [ ]:
# Importación de bibliotecas necesarias if you want to download the dataset from kaggle
#import kagglehub
#import shutil

# Descarga del conjunto de datos de Kaggle
# Este conjunto contiene imágenes de tomografías computarizadas del cerebro
# con casos de ictus y casos normales
#path = kagglehub.dataset_download("afridirahman/brain-stroke-ct-image-dataset")
#print("Ruta a los archivos del dataset:", path)

# Copia de los datos descargados a una ubicación local
# Esto nos permite trabajar con los datos sin necesidad de volver a descargarlos
#src = path  # Usamos la variable path que contiene la ubicación de descarga
#dst = r'.\Dataset_kaggle'

#shutil.copytree(src, dst)
#print(f"Dataset copiado de {src} a {dst}")

In [1]:
# Importación de bibliotecas necesarias para el procesamiento de datos
import os
import shutil
import random
from pathlib import Path

# Configuración de directorios y parámetros
source_dir = '../../data/Brain_Data_Organised'  # Directorio fuente con las imágenes originales
target_dir = 'Dataset_img_for_CNN'  # Directorio donde organizaremos los datos para el entrenamiento
split_ratio = 0.8  # Proporción 80% para entrenamiento, 20% para prueba
classes = ['Stroke', 'Normal']  # Las dos clases que queremos clasificar

# Creación de la estructura de directorios para el entrenamiento
print("📁 Creando directorios de destino...")
for phase in ['train', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(target_dir, phase, cls), exist_ok=True)

# Configuración de la semilla aleatoria para reproducibilidad
random.seed(42)  # Esto asegura que obtengamos los mismos resultados cada vez que ejecutemos el código

# Procesamiento de cada clase
for cls in classes:
    cls_dir = os.path.join(source_dir, cls)
    
    print(f"\n🔄 Procesando clase: {cls}")
    print(f"   Buscando archivos en: {cls_dir}")
    
    # Verificación de la existencia del directorio
    if not os.path.exists(cls_dir):
        print(f"❌ ¡Carpeta {cls_dir} no encontrada!")
        continue
    
    # Búsqueda de imágenes en formatos comunes
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.tif']
    images = []
    
    # Recopilación de todas las imágenes disponibles
    for ext in image_extensions:
        images.extend(list(Path(cls_dir).glob(ext)))
        images.extend(list(Path(cls_dir).glob(ext.upper())))  # También en mayúsculas
    
    print(f"   Imágenes encontradas: {len(images)}")
    
    if len(images) == 0:
        print(f"⚠️  No hay imágenes en la carpeta {cls_dir}")
        continue
    
    # División aleatoria de las imágenes
    random.shuffle(images)
    split_idx = int(len(images) * split_ratio)
    train_images = images[:split_idx]
    test_images = images[split_idx:]
    
    print(f"   División: {len(train_images)} para entrenamiento, {len(test_images)} para prueba")
    
    # Copia de archivos a sus respectivos directorios
    for i, img_path in enumerate(train_images):
        try:
            dest_path = os.path.join(target_dir, 'train', cls, img_path.name)
            shutil.copy2(img_path, dest_path)
            if i == 0:  # Imprimir el primer archivo para verificación
                print(f"   ✅ Primer archivo de entrenamiento: {img_path.name}")
        except Exception as e:
            print(f"   ❌ Error al copiar {img_path.name}: {e}")
    
    for i, img_path in enumerate(test_images):
        try:
            dest_path = os.path.join(target_dir, 'test', cls, img_path.name)
            shutil.copy2(img_path, dest_path)
            if i == 0:  # Imprimir el primer archivo para verificación
                print(f"   ✅ Primer archivo de prueba: {img_path.name}")
        except Exception as e:
            print(f"   ❌ Error al copiar {img_path.name}: {e}")

# Verificación de resultados
print("\n" + "="*50)
print("📊 Estadísticas finales:")
for phase in ['train', 'test']:
    for cls in classes:
        path = os.path.join(target_dir, phase, cls)
        if os.path.exists(path):
            count = len(os.listdir(path))
            print(f"   {phase}/{cls}: {count} archivos")

print("\n✅ ¡Distribución completada!")

📁 Creando directorios de destino...

🔄 Procesando clase: Stroke
   Buscando archivos en: ../../data/Brain_Data_Organised\Stroke
   Imágenes encontradas: 1900
   División: 1520 para entrenamiento, 380 para prueba
   ✅ Primer archivo de entrenamiento: 67 (14).jpg
   ✅ Primer archivo de prueba: 75 (24).jpg

🔄 Procesando clase: Normal
   Buscando archivos en: ../../data/Brain_Data_Organised\Normal
   Imágenes encontradas: 2338
   División: 1870 para entrenamiento, 468 para prueba
   ✅ Primer archivo de entrenamiento: 54 (7).jpg
   ✅ Primer archivo de prueba: 51 (16).jpg

📊 Estadísticas finales:
   train/Stroke: 913 archivos
   train/Normal: 1122 archivos
   test/Stroke: 343 archivos
   test/Normal: 421 archivos

✅ ¡Distribución completada!


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## 2. Аumentación

La aumentación de datos es un proceso muy potente que permite aumentar la cantidad de datos de entrenamiento. Mediante rotaciones, reflejos, adición de ruido, desplazamientos y otras transformaciones, la imagen cambia ligeramente, pero mantiene su etiqueta original. Con la función Compose podemos combinar varias transformaciones de imagen y luego aplicarlas al leer el conjunto de datos. La lista completa de aumentaciones está disponible [aquí](https://pytorch.org/vision/stable/transforms.html). Estúdiala y experimenta con diferentes transformaciones de imagen.

Una herramienta bastante potente y eficiente para la aumentación de imágenes es la biblioteca `albumentations`.

In [ ]:
# Transformaciones para el conjunto de entrenamiento
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar todas las imágenes a 224x224 píxeles
    transforms.Grayscale(num_output_channels=3),  # Convertir a escala de grises pero mantener 3 canales
    transforms.RandomHorizontalFlip(),  # Volteo horizontal aleatorio para aumentar la variedad
    transforms.RandomRotation(5),  # Rotación aleatoria de hasta 5 grados
    transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Variación aleatoria de brillo y contraste
    transforms.ToTensor(),  # Convertir imagen a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalización de los valores de píxeles
])

# Transformaciones para el conjunto de validación y prueba
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Mismo tamaño que el conjunto de entrenamiento
    transforms.Grayscale(num_output_channels=3),  # Misma conversión a escala de grises
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Misma normalización
])

# Load the full training dataset
train_dataset = datasets.ImageFolder(
    root='Dataset_img_for_CNN/train',
    transform=train_transforms
)

test_dataset = datasets.ImageFolder(
    root='Dataset_img_for_CNN/test',
    transform=val_test_transforms
)

# Split training data into train and validation sets (80-20 split)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

# Use random_split to create train and validation datasets
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create validation dataset with appropriate transforms
val_dataset.dataset = datasets.ImageFolder(
    root='Dataset_img_for_CNN/train',
    transform=val_test_transforms
)
val_dataset = torch.utils.data.Subset(val_dataset.dataset, val_dataset.indices)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 1928
Validation samples: 483
Test samples: 911


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



## 3. Regularización y normalización en redes neuronales

### Dropout
Si la red tiene una arquitectura compleja, es posible el sobreajuste (overfitting) - un proceso en el que el modelo se adapta demasiado a los datos de entrenamiento y luego da un rendimiento inferior en los datos de prueba. Para combatir esto, se puede utilizar Dropout. La idea del método es muy simple. Durante el entrenamiento, `torch.nn.Dropout` establece a cero cada elemento del tensor de entrada con una probabilidad $p$. Durante la inferencia, no se establece nada a cero, pero para mantener la escala de las salidas de la red, todos los elementos del tensor de entrada se dividen por $1 - p$.

![Dropout](https://github.com/hse-ds/iad-deep-learning/blob/master/2022/seminars/sem03/static/dropout.png?raw=1)

Para estabilizar y acelerar la convergencia del entrenamiento, se utiliza frecuentemente la normalización por lotes (batch normalization). En **PyTorch** también está implementada como una capa — [`torch.nn.BatchNorm2d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html). Generalmente, la normalización por lotes se inserta entre los bloques significativos de la red neuronal para mantener la distribución de los datos durante todo el forward pass. Tenga en cuenta que durante el entrenamiento, la media y la desviación estándar muestral se calculan de nuevo para cada lote, y la capa tiene dos parámetros numéricos entrenables para cada canal del tensor de entrada. Durante la inferencia, se utilizan como media y varianza las estimaciones obtenidas mediante promedios móviles durante el entrenamiento.

![Batch Norm](https://github.com/hse-ds/iad-deep-learning/blob/master/2022/seminars/sem03/static/batch_norm.png?raw=1)


![Typical CNN architecture](Typical%20CNN%20architecture.png)


In [ ]:
# Definición de la arquitectura CNN
model = nn.Sequential(
    # Primera capa convolucional
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # 3 canales de entrada, 32 filtros
    nn.BatchNorm2d(32),  # Normalización por lotes
    nn.ReLU(),  # Función de activación
    nn.MaxPool2d(2, 2),  # Reducción de dimensionalidad

    # Segunda capa convolucional
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # 32 canales de entrada, 64 filtros
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    # Tercera capa convolucional
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # 64 canales de entrada, 128 filtros
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    # Capas fully connected
    nn.Flatten(),  # Aplanar la salida para las capas densas
    nn.Dropout(0.1),  # Regularización para evitar overfitting
    nn.Linear(128*28*28, 512),  # Primera capa densa
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(512, 2)  # Capa de salida (2 clases)
)

# Set device and move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Using device: {device}")

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

Using device: cuda


### Class torch.nn.Conv2d

 
En **PyTorch**, la capa convolucional está representada en el módulo `torch.nn` por la clase [`Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) con los siguientes parámetros:
- `in_channels`: número de canales de entrada
- `out_channels`: número de canales de salida
- `kernel_size`: tamaño del kernel (núcleo)
- `stride`: paso (desplazamiento)
- `padding`: relleno
- `padding_mode`: modo de relleno (`'zeros'`, `'reflect'` y otros)
- `dilation`: dilatación

#### `kernel_size`

**Tamaño del kernel (núcleo)**. `int`, si el kernel es cuadrado, y una tupla de dos números si el kernel es rectangular. Define el tamaño del filtro con el que se realiza la convolución de la imagen.

**`kernel_size=3`**

![no_padding_no_strides.gif](static/no_padding_no_strides.gif)

Esta y las siguientes animaciones están tomadas de [aquí](https://github.com/vdumoulin/conv_arithmetic).

#### `stride`

**Paso (stride)**. Define el paso, en píxeles, con el que se desplaza el filtro. `int`, si el desplazamiento es el mismo en horizontal y vertical. Una tupla de dos números, si los desplazamientos son diferentes.

**`stride=2`**

![no_padding_strides.gif](static/no_padding_strides.gif)

#### `padding`

**Relleno (padding)**. Cantidad de píxeles con los que se complementa la imagen. Similar al paso y al tamaño del kernel, puede ser tanto `int` como una tupla de dos números.

**`padding=1`**

![same_padding_no_strides.gif](static/same_padding_no_strides.gif)

In [ ]:
# Training parameters
num_epochs = 20

# Lists to store training history
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

print("Starting training...")
print("=" * 70)

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    print(f'Epoch [{epoch+1}/{num_epochs}]')

    # Training batches
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Move data to device
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate training metrics
    train_accuracy = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)

    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation phase
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            # Move data to device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Statistics
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate validation metrics
    val_accuracy = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)

    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Print epoch results
    print(f'  Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'  Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
    print('-' * 70)

print('Training completed!')
print("=" * 70)


Starting training...
Epoch [1/20]
  Train Loss: 0.8707, Train Accuracy: 65.35%
  Val Loss: 0.4867, Val Accuracy: 73.29%
----------------------------------------------------------------------
Epoch [2/20]
  Train Loss: 0.5155, Train Accuracy: 74.90%
  Val Loss: 0.4168, Val Accuracy: 78.05%
----------------------------------------------------------------------
Epoch [3/20]
  Train Loss: 0.5135, Train Accuracy: 74.12%
  Val Loss: 0.3637, Val Accuracy: 80.75%
----------------------------------------------------------------------
Epoch [4/20]
  Train Loss: 0.4448, Train Accuracy: 77.54%
  Val Loss: 0.3550, Val Accuracy: 81.37%
----------------------------------------------------------------------
Epoch [5/20]
  Train Loss: 0.3351, Train Accuracy: 83.71%
  Val Loss: 0.2728, Val Accuracy: 86.13%
----------------------------------------------------------------------
Epoch [6/20]
  Train Loss: 0.2782, Train Accuracy: 87.76%
  Val Loss: 0.2445, Val Accuracy: 88.20%
------------------------------

In [ ]:
# Final evaluation on test set
print("Evaluating on test set...")
model.eval()

# Collect all predictions and true labels for classification report
all_predictions = []
all_labels = []
correct_test = 0
total_test = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        # Move data to device
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Statistics
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

        # Collect predictions and labels for classification report
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate final test metrics
test_accuracy = 100 * correct_test / total_test
avg_test_loss = test_loss / len(test_loader)

print(f"\nFINAL TEST RESULTS:")
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print("=" * 70)

# Generate classification report
class_names = test_dataset.classes
print("\nDETAILED CLASSIFICATION REPORT:")
print(classification_report(all_labels, all_predictions,
                          target_names=class_names,
                          digits=4))

# Print training summary
print("\nTRAINING SUMMARY:")
print(f"Best validation accuracy: {max(val_accuracies):.2f}% (Epoch {val_accuracies.index(max(val_accuracies))+1})")
print(f"Final validation accuracy: {val_accuracies[-1]:.2f}%")
print(f"Final test accuracy: {test_accuracy:.2f}%")
print("=" * 70)

Evaluating on test set...

FINAL TEST RESULTS:
Test Loss: 0.0618
Test Accuracy: 98.13%

DETAILED CLASSIFICATION REPORT:
              precision    recall  f1-score   support

      normal     0.9946    0.9754    0.9849       568
      stroke     0.9605    0.9913    0.9756       343

    accuracy                         0.9813       911
   macro avg     0.9775    0.9833    0.9802       911
weighted avg     0.9818    0.9813    0.9814       911


TRAINING SUMMARY:
Best validation accuracy: 96.69% (Epoch 13)
Final validation accuracy: 96.07%
Final test accuracy: 98.13%


In [ ]:

# Save the final trained model
import os

# Create directory for saving models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Save the complete final model
model_path = f"{save_dir}/maryna_cnn_model.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'final_train_accuracy': train_accuracies[-1],
    'final_val_accuracy': val_accuracies[-1],
    'test_accuracy': test_accuracy,
    'num_epochs': num_epochs,
    'model_architecture': str(model),
    'class_names': class_names,
    'training_history': {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies
    }
}, model_path)

print(f"Model saved to: {model_path}")

print("\n" + "="*50)
print("MODEL LOADING INSTRUCTIONS:")
print("="*50)
print("To load the model:")
print("checkpoint = torch.load('saved_models/maryna_cnn_model.pth')")
print("model.load_state_dict(checkpoint['model_state_dict'])")
print("test_accuracy = checkpoint['test_accuracy']")
print("="*50)

Model saved to: saved_models/maryna_cnn_model.pth

MODEL LOADING INSTRUCTIONS:
To load the model:
checkpoint = torch.load('saved_models/maryna_cnn_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
test_accuracy = checkpoint['test_accuracy']


In [ ]:
# Code to load the saved model

import torch
import torch.nn as nn

# Recreate the model architecture (must be identical to training)
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),
    nn.Dropout(0.1),
    nn.Linear(128*28*28, 512),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(512, 2)
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved checkpoint
checkpoint = torch.load('model_maryna.pth', map_location=device)

# Load the model weights
model.load_state_dict(checkpoint['model_state_dict'])

# Move model to device
model = model.to(device)

# Set model to evaluation mode
model.eval()

# Access saved information
test_accuracy = checkpoint['test_accuracy']
class_names = checkpoint['class_names']
training_history = checkpoint['training_history']

print(f"Model loaded successfully!")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Classes: {class_names}")
print(f"Training completed in {checkpoint['num_epochs']} epochs")